# Data preparation

In [25]:
import pandas as pd
import numpy as np 

In [2]:
path = '../data/' 
hotels = pd.read_csv(path + '/features_hotels.csv')
data = pd.read_csv(path + 'data.csv')
data_test = pd.read_csv(path + 'test_set.csv')

In [3]:
to_exclude = data[['city', 'language','mobile','request_number','date']].drop_duplicates()
# to_exclude.to_csv('to_exclude.csv', index = False)
print('Nombre de requêtes uniques :', to_exclude.shape[0])

Nombre de requêtes uniques : 8329


In [4]:
tot = 0 
for i in range(7):
    tot += np.unique(data.loc[data.request_number == i].avatar_id.values).shape[0]

print('Estimation du nombre total de requêtes :', tot)

Estimation du nombre total de requêtes : 11171


In [5]:
# exclusion des avatar posant problème
# data = data.drop(index = data.loc[data.avatar_id == 134].index)
# to_drop = [108,134,135,136,137,138,141]
to_drop = [134]
data = data.loc[~data.avatar_id.isin(to_drop)]

In [6]:
data = data.drop(['avatar_id'], axis = 1).drop_duplicates()

In [7]:
# ajout des caractéristiques des hotels
data = data.merge(hotels, on=['hotel_id','city'])
data_test = data_test.merge(hotels, on=['hotel_id','city'])
data_test = data_test.sort_values('index').reset_index(drop=True).drop(['index'], axis = 1)

In [8]:
# création de la colonne request_number dans le test set
data_test['request_number'] = 1
for avatar in np.unique(data_test['avatar_id']):
    data_test.loc[data_test['avatar_id'] == avatar, 'request_number'] = data_test['order_requests'].loc[data_test['avatar_id']== avatar] - min(data_test['order_requests'].loc[data_test['avatar_id']== avatar])+1

In [9]:
# ajout de la variable ville == langue
lang_cit = True

if lang_cit == True: 
    dic_lang = {'amsterdam':'dutch', 'copenhagen':'danish', 'madrid':'spanish', 'paris':'french', 'rome':'italian', 'sofia':'bulgarian', 'valletta':'maltese', 'vienna':'austrian' ,'vilnius':'lithuanian'}
    data['city_language'] = data['city'].map(dic_lang)
    data['is_same_cl'] = data['city_language']==data['language']
    data_test['city_language'] = data_test['city'].map(dic_lang)
    data_test['is_same_cl'] = data_test['city_language']==data_test['language']
#     colBool += ['is_same_cl']

In [10]:
col = ['city', 'date', 'language', 'mobile', 'request_number', 'stock', 'group', 'brand', 'parking', 'pool','children_policy', 'is_same_cl', 'price']
data = data[col]
data_test = data_test[col[:-1]]

In [11]:
def additive_smoothing(x,alpha= 1.96):
    """ alpha = 1.96 borne à 95% pour la loi normale  """
    moy = (x+alpha).mean()
    d = x.max()-x.min()
    N = x.count()
    return N*moy/(N+alpha*d)

additive_smoothing.__name__ = 'additive_smoothing'

In [12]:
colQT = ['request_number', 'stock', 'date']
notQT = ['city', 'language', 'mobile', 'group', 'brand', 'parking', 'pool','children_policy', 'is_same_cl'] 

In [13]:
for i in notQT:
    df_tmp = data[[i, 'price']].groupby(i).agg({'price': ['mean', 'var', additive_smoothing]}).price
    df_tmp = df_tmp.add_suffix('_' + i)
    colQT += list(df_tmp.columns.unique())
    data = data.join(df_tmp, on=i)
    data_test = data_test.join(df_tmp, on=i)

In [14]:
colQT += ['price']

In [15]:
df = data[colQT[:-1]]
df_test = data_test[colQT[:-1]]
y = data[colQT[-1]]

In [16]:
X_train = df
X_test = df_test
Y_train = y
Y_test = None

In [ ]:
# pd.read_csv('test.csv')

# GBR

In [17]:
X_train

,request_number,stock,date,mean_city,var_city,additive_smoothing_city,mean_language,var_language,additive_smoothing_language,mean_mobile,...,additive_smoothing_parking,mean_pool,var_pool,additive_smoothing_pool,mean_children_policy,var_children_policy,additive_smoothing_children_policy,mean_is_same_cl,var_is_same_cl,additive_smoothing_is_same_cl
0,1,3,12,165.55173,5985.767282,166.516354,154.748637,5999.095160,150.843929,159.610475,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
1,1,15,27,165.55173,5985.767282,166.516354,159.636337,6808.807107,158.623704,160.026551,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
2,1,2,11,165.55173,5985.767282,166.516354,156.157476,6641.903925,155.367276,159.610475,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
3,1,14,26,165.55173,5985.767282,166.516354,158.518461,6578.472846,154.689115,160.026551,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
4,1,13,25,165.55173,5985.767282,166.516354,156.713749,6541.710854,152.944919,159.610475,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916700,1,84,39,165.55173,5985.767282,166.516354,164.508062,7283.720668,159.483584,159.610475,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
916701,1,84,39,165.55173,5985.767282,166.516354,156.456098,6441.953525,153.253943,160.026551,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
916702,1,110,44,165.55173,5985.767282,166.516354,165.404823,7131.140515,164.022882,160.026551,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872
916703,1,104,43,165.55173,5985.767282,166.516354,155.846236,6642.503216,152.507311,159.610475,...,162.905065,142.144634,4905.884746,143.917897,157.313797,6680.118848,159.092193,160.002779,6871.253349,161.778872


In [19]:
from sklearn.ensemble import GradientBoostingRegressor

GBR = GradientBoostingRegressor()

GBR.set_params(learning_rate = .02,
               max_depth = 4,
               n_estimators = 5000,
               validation_fraction = .2,
               criterion = 'friedman_mse',
               subsample = .2,
               max_leaf_nodes = 50,
               max_features = 1.0,
               verbose = 1)

fit = GBR.fit(X_train,Y_train)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1        6625.9880         236.7315           28.62m
         2        6400.5810         227.8382           28.36m
         3        6127.9613         219.1769           28.62m
         4        5956.9103         210.7144           28.98m
         5        5818.9556         201.8372           28.97m
         6        5541.5158         194.1658           28.92m
         7        5405.4958         186.8154           28.78m
         8        5170.4459         179.7322           28.67m
         9        5002.3840         173.2711           28.72m
        10        4856.5649         165.7229           28.61m
        20        3495.3357         112.9255           28.75m
        30        2594.7882          76.7821           28.86m
        40        1971.0378          52.6011           28.87m
        50        1533.6797          36.2118           28.77m
        60        1235.7556          25.5393           28.69m
       

In [ ]:
# GBR.get_params().keys()

In [20]:
y_pred = fit.predict(X_test)

In [21]:
y_pred

array([105.2317202 , 135.68128139,  69.41514648, ..., 156.71843755,
       178.27597186, 156.71843755])

In [37]:
# from joblib import dump, load
# dump(GBR, 'model/target_encoding.pkl')

# xgboost classique

In [ ]:
import xgboost as xgb

def custom_loss(preds, dtrain):
    labels = dtrain.get_label()
    return 'custom_loss', np.sqrt(np.mean((preds - labels)**2))

# Charger les données d'apprentissage et de test
X_train, y_train = df_RF , y
X_test, y_test = df_RF_test , None

# Convertir les données en format DMatrix pour XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Définir les paramètres du modèle
param = {'max_depth': 4,
         'eta': 0.5, # learning rate
         'gamma': 1,
         'min_child_weight': 1,
         'subsample': 0.2,
         'max_leaves': 10,
         'objective': 'reg:squarederror'}

# Entraîner le modèle
model = xgb.train(param, dtrain, num_boost_round=10) #, custom_metric=custom_loss)

# Faire des prévisions sur les données de test
predictions = model.predict(dtest)

# Afficher les prévisions
print(predictions)

# Cross Validation

In [ ]:
import xgboost as xgb

def custom_loss(preds, dtrain):
    labels = dtrain.get_label()
    return 'custom_loss', np.sqrt(np.mean((preds - labels)**2))

# Charger les données
X, y = X_train, Y_train

# Convertir les données en format DMatrix pour XGBoost
dmatrix = xgb.DMatrix(X, label=y)

# Définir les paramètres du modèle
param = {'max_depth': 4,
         'eta': 0.5, # learning rate
         'gamma': 1,
         'min_child_weight': 1,
         'subsample': 0.2,
         'max_leaves': 10,
         'objective': 'reg:squarederror'}

# Définir les paramètres de la validation croisée
cv_params = {'num_boost_round': 10, 'nfold': 5, 'metrics': 'rmse', 'seed': 0}

# Entraîner le modèle en utilisant la validation croisée
cv_results = xgb.cv(param, dmatrix, **cv_params)

# Afficher les résultats de la validation croisée
print(cv_results)

# GridSearch

In [45]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

X, y = X_train, Y_train
dmatrix = xgb.DMatrix(X, label=y)

param_grid = {
    'max_depth': [4],
    'gamma' : [2.5],
    'subsample': [.23],
    'eta': np.linspace(.01,.115,10), # learning rate
    'colsample_bytree':np.linspace(.001,1,10),
    'objective': ['reg:squarederror'],
}

cv_params = {'num_boost_round': 15, 'nfold': 6, 'metrics': 'rmse', 'seed': 0}

xgb_model = xgb.XGBModel(**cv_params)

random_search = RandomizedSearchCV(xgb_model,
                                   param_grid,
                                   n_iter=5,
                                   cv=6,
                                   scoring='accuracy',
                                   random_state=0,
                                   verbose=True)

random_search.fit(X, y)

print(random_search.best_params_)

/usr/local/insa/anaconda/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Fitting 6 folds for each of 5 candidates, totalling 30 fits
[17:45:49] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:576: 
Parameters: { "metrics", "nfold", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 219, in __call__
    return self._score(
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metric

[17:45:57] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:576: 
Parameters: { "metrics", "nfold", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 219, in __call__
    return self._score(
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metric

...

[17:49:26] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:576: 
Parameters: { "metrics", "nfold", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 219, in __call__
    return self._score(
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metric

[17:49:35] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:576: 
Parameters: { "metrics", "nfold", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 219, in __call__
    return self._score(
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metric

[17:49:45] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:576: 
Parameters: { "metrics", "nfold", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


{'subsample': 0.23, 'objective': 'reg:squarederror', 'max_depth': 4, 'gamma': 2.5, 'eta': 0.08, 'colsample_bytree': 0.223}


In [50]:
# Charger les données de test
X_test, y_test = X_test, Y_test

# Faire une prédiction avec le modèle XGBoost entraîné
predictions = random_search.predict(X_test)

# Afficher les prédictions
print(predictions)

[111.76069 134.29666  66.89325 ... 138.23439 167.6618  138.23439]


/usr/local/insa/anaconda/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [51]:
predictions

array([111.76069, 134.29666,  66.89325, ..., 138.23439, 167.6618 ,
       138.23439], dtype=float32)

# Optuna

In [ ]:
import optuna
import xgboost as xgb
    
def objective(trial):
    # Charger les données
    X, y = X_train, Y_train

    # Convertir les données en format DMatrix pour XGBoost
    dmatrix = xgb.DMatrix(X, label=y)

    # Définir les paramètres du modèle
    param = {
        'max_depth': trial.suggest_int('max_depth', 2, 8),
        'eta': trial.suggest_uniform('eta', 0.01, 0.4),
        'subsample': trial.suggest_uniform('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'objective': 'reg:squarederror',
    }

    # Définir les paramètres de la validation croisée
    cv_params = {'num_boost_round': 10, 'nfold': 5, 'metrics': 'rmse', 'seed': 7}

    # Entraîner le modèle en utilisant la validation croisée
    cv_results = xgb.cv(param, dmatrix, **cv_params)

    print(cv_results['test-rmse-mean'])

    # Récupérer la métrique de performance
    mean_error = cv_results['test-rmse-mean'].min()
    
    # Inverser la métrique pour l'optimiser
    return 1.0 - mean_error

# Définir l'étude Optuna
study = optuna.create_study()

# Rechercher les meilleurs paramètres en utilisant Optuna
study.optimize(objective, n_trials=10)

# Afficher les meilleurs paramètres
print(study.best_params)

In [ ]:
y_pred = pd.read_csv('a.csv')

In [ ]:
predictions = y_pred+28

In [ ]:
# Convertir les données en format DMatrix pour XGBoost
dmatrix_test = xgb.DMatrix(X_test, label = y_pred.price)
xgb_model = xgb.train(study.best_params, dmatrix_test)
y_pred = xgb_model.predict(dmatrix_test)

In [ ]:
import optuna

# Définir l'étude Optuna
study = optuna.create_study()

# Rechercher les meilleurs paramètres en utilisant Optuna
study.optimize(objective, n_trials=10)

# Enregistrer l'étude dans un fichier
study.trials_dataframe().to_pickle('example.pkl')

In [ ]:
import optuna

# Charger l'étude à partir d'un fichier
study = optuna.load_study(study_name='my_study', storage='sqlite:///example.db')

# Continuer la recherche de paramètres en utilisant Optuna
study.optimize(objective, n_trials=10)

# Afficher les meilleurs paramètres
print(study.best_params)

In [ ]:
 X, y = X_train, Y_train

# Convertir les données en format DMatrix pour XGBoost
dmatrix = xgb.DMatrix(X, label=y)

# Définir les paramètres du modèle
param = {}

# Définir les paramètres de la validation croisée
cv_params = {'num_boost_round': 10, 'nfold': 5, 'metrics': 'rmse', 'seed': 7}

# Entraîner le modèle en utilisant la validation croisée
cv_results = xgb.cv(param, dmatrix, **cv_params)


xgb.cv(param, dmatrix, **cv_params)

# Submit

In [47]:
predictions = y_pred

In [52]:
# save
_round = False # tester aussi des arrondis au dessus ou en dessous

name = 'grid_search'

if _round : y_pred = np.round(predictions)
else : y_pred = predictions.copy()

sub = pd.DataFrame(y_pred)
sub.to_csv(path + 'submit/' + name + '.csv',index=True, header=['price'], index_label = 'index')

In [ ]:
# en dessous on fait la moyenne de plusieurs fichiers d'un dossier 

In [ ]:
import pandas as pd
import os

def create_dataframe(directory_path):
    # Initialiser le dataframe
    df = pd.DataFrame()

    # Pour chaque fichier dans le dossier
    for file_name in os.listdir(directory_path):
        # Charger les données du fichier
        file_data = pd.read_csv(os.path.join(directory_path, file_name))

        # Ajouter une colonne
        df[file_name] = file_data.price
    return df

# Utiliser la fonction
df = create_dataframe('../data/best_sub/')

In [ ]:
df

In [ ]:
# Calculer la moyenne de chaque ligne
mean_by_row = df.mean(axis=1)

In [ ]:
predictions = mean_by_row

In [ ]:
X_train.to_csv('train.csv',index=True)

In [ ]:
X_test.to_csv('test.csv',index=True)